In [ ]:
%%html
<marquee style='width: 30%; color: blue;'><b>5430 NLP | SPRING 2021 | ASSIGNMENT 9 | UNI: CHB2132</b></marquee>

#`5430 NLP | SPRING 2021 | ASSIGNMENT 9 | UNI: CHB2132 `#

---

###**Write a Python program, which:**

**1. Implements a keyword or sentence taxonomy of topic classes.**

**Notes:**
* Derive taxonomy from: LDA topic clustering results and some amount of manual enrichment
* Taxonomy from class may not generalize over your specific dataset: do not use!
* Best with Word2Vec: Keyword-Based Taxonomy
* Best with BERT: Sentence-Based Taxononomy

**2. Classifies Webhose titles against developed taxonomy using: semantic_text_similarity (BERT) or word2vec similarity and groups them by topic classes.**

**Final submission should include:**
1. Jupyter (.ipynb) or Python (.py) notebook with your implementation
2. Explicit JSON representation of your taxonomy 
3. Output showing each topic in taxonomy and top 10 matching articles

(https://courseworks2.columbia.edu/courses/129128/assignments/591463?module_item_id=1211044)

#### *Load Pre-trained Model, Packages, etc.*
---

In [ ]:
#!pip install semantic-text-similarity

#load_ext google.colab.data_table
%reload_ext google.colab.data_table

In [ ]:
import json
import numpy as np
import pandas as pd

import gensim, operator
from gensim.models import KeyedVectors

from scipy import spatial

model_path = '/content/'

### *Import set of deduplicated Webhose Apple News titles (JSON)*
---

In [ ]:
# import deduplicated newsfeed dataset

title = []

json_data = open('/content/dedupes.json').readlines()

for line in json_data:
    title.append(json.loads(line))

news_feeds = pd.DataFrame(title)
title_df = pd.DataFrame(news_feeds['title'])

### *Load in Google Word2Vec Model*
---

In [ ]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True)

Loading Word2Vec model...


EOFError: ignored

In [ ]:
def vocab_check(vectors, words):
    output = []
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
    return output

#### *Implement a keyword or sentence taxonomy of topic classes*
---

In [ ]:
# custom taxonomy for apple webhose newsfeed data vocab/semantics

topic_taxonomy = {
    'News':
	    {
            'Looters' : 'iPhone Looters Being Tracked - Apple Warns iPhone Looters',
            'Hackers' : 'Apple bug exposed user accounts to hackers',
            'Stolen' : 'Apple Warns Looters - The iPhones You Stole Are Tracked',
            'Service Outage' : 'iCloud and other Apple services experienced an outage',
            'Outage Impacts' : 'iCloud outage impacts Mail, web apps, more'
        },

    'iOS':
        {
            'iPhone' : 'iOS 14 Will Reportedly Support All iPhone Models Running iOS 13',
            'iCloud' : 'iCloud outage took down some macOS and iOS software',
            'Safari' :  'Safari in iOS and iPadOS 14 Might Include Built-In Translation, Better',
            'AirDrop' : "Sharedrop - HTML5 clone of Apple's AirDrop",
            'YouTube Kids' : 'Apple TV Users Can Now Enjoy YouTube Kids'
        },

    'Apple':
        {
            'New' : 'New iPad Air may come with USB-C not Lightning Port',
            'Price' : 'Deutsche Bank raises Apple price target $15 to $320',
            'Sales' : 'iPhone Sales Took a Hit Due to Coronavirus',
            'Cloud': 'iCloud AirDrop AppleTV iTunes storage photos appleID icloud.com',
            'CEO' : 'Tech investor urges Apple CEO Tim Cook, Chief Executive Officer'
        },

    'App Store':
        {
            'Apple TV':'Peloton releases dedicated Apple TV app with thousands of workouts',
            'StopCovid': "French govt's StopCovid tracing app",
            'Apple Pay': "Why are Apple Pay, Starbucks' app, and Samsung"
        }
}

#### *Calculate Semantic Similarity of 2 Strings*
---

In [ ]:
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    output = vectors.n_similarity(s1words, s2words)
    return output

#### *Classify titles by topics using keyword max score value*
---

In [ ]:
def classify_topics(input, vectors):
    score = []
    feed_score = dict()

    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))

        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
        

    return (sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:1])

#### *Output each taxonomy topic and top 10 matching articles*
---

In [ ]:
f = 0
f_max = len(title_df)

tts_list = []
score_list = []
title_list = []
topic_list = []

for f in range(f_max):

    title = str(title_df['title'][[f]])
    title_list.append(title)
    title_low = str(title.lower)

    ts_list = classify_topics(title_low, model)
    
    topic = ts_list[0][0]
    score = ts_list[0][1]

    topic_list.append(topic)
    score_list.append(score)

In [ ]:
title_df = pd.DataFrame(data = title_list, columns = ['title'])
topic_df = pd.DataFrame(data = topic_list, columns = ['topic'])
score_df = pd.DataFrame(data = score_list, columns = ['score'])

tts_df = pd.concat([title_df, topic_df, score_df], names = ['title', 'topic', 'score'], axis = 1)
top_df = tts_df.sort_values(['topic', 'score'], ascending = True).head(10)

top_df

#### *Output explicit JSON representation of taxonomy*
---

In [ ]:
with open("taxonomy.json", "w") as data_file:
  
    for topic in topic_taxonomy:
        line = json.dumps(topic)
        data_file.write(line)
        data_file.write("\n")

json_data = open("taxonomy.json").readlines()

taxonomy_read = []

for line in json_data:
    taxonomy_read.append(json.loads(line))

taxonomy_read